<a href="https://colab.research.google.com/github/Narennnnn/EmoNet-Deep-Learning-for-Facial-Expression-Recognition/blob/main/facialExpressionDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
ls


drive/  sample_data/


In [ ]:
%cd drive/MyDrive/Facial\ Expression\ Dataset/

/content/drive/MyDrive/Facial Expression Dataset


In [ ]:
!unzip Dataset.zip

Archive:  Dataset.zip
  inflating: fer2013.csv             
  inflating: fer2013/fer2013.csv     


In [ ]:
import pandas as pd

csv_path = '/content/drive/MyDrive/Facial Expression Dataset/fer2013.csv'
data = pd.read_csv(csv_path)

image_paths = data['pixels'].values
labels = data['emotion'].values


In [ ]:
# Process the image paths
preprocessed_images = []
for image_data in image_paths:
    image = np.array(image_data.split(' '), dtype=np.uint8)
    image = image.reshape((48, 48))
    preprocessed_images.append(image)


In [ ]:
# Convert the preprocessed data to NumPy arrays
preprocessed_images = np.array(preprocessed_images)
labels = np.array(labels)


In [ ]:
# Convert the preprocessed data to NumPy arrays
preprocessed_images = np.array(preprocessed_images)
labels = np.array(labels)

In [ ]:
# Data augmentation using ImageDataGenerator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
# Generate augmented images and labels
augmented_images = []
augmented_labels = []
for image, label in zip(preprocessed_images, labels):
    image = np.expand_dims(image, axis=-1)  # Add an additional dimension for the channel
    image = np.expand_dims(image, axis=0)  # Add an additional dimension for the batch
    aug_iter = datagen.flow(image, save_to_dir=None, shuffle=False)
    for aug_image in aug_iter:
        augmented_images.append(aug_image[0])
        augmented_labels.append(label)
        break

augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

In [ ]:
# Train-validation-test split
X_train, X_test, y_train, y_test = train_test_split(augmented_images, augmented_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# Build the deep learning model
num_classes = len(np.unique(labels))
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=10000, decay_rate=0.9
)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


Epoch 1/10
718/718 [==============================] - 274s 379ms/step - loss: 1.9800 - accuracy: 0.2777 - val_loss: 1.6648 - val_accuracy: 0.3532
Epoch 2/10
718/718 [==============================] - 278s 387ms/step - loss: 1.6232 - accuracy: 0.3629 - val_loss: 1.5923 - val_accuracy: 0.3811
Epoch 3/10
718/718 [==============================] - 278s 387ms/step - loss: 1.5241 - accuracy: 0.4079 - val_loss: 1.4992 - val_accuracy: 0.4241
Epoch 4/10
718/718 [==============================] - 277s 386ms/step - loss: 1.4377 - accuracy: 0.4462 - val_loss: 1.4497 - val_accuracy: 0.4342
Epoch 5/10
718/718 [==============================] - 265s 370ms/step - loss: 1.3477 - accuracy: 0.4870 - val_loss: 1.4517 - val_accuracy: 0.4458
Epoch 6/10
718/718 [==============================] - 261s 363ms/step - loss: 1.2524 - accuracy: 0.5245 - val_loss: 1.4708 - val_accuracy: 0.4589
Epoch 7/10
718/718 [==============================] - 270s 376ms/step - loss: 1.1455 - accuracy: 0.5682 - val_loss: 1.4870 -

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

225/225 [==============================] - 31s 138ms/step - loss: 2.0053 - accuracy: 0.4317
Test Loss: 2.0053277015686035
Test Accuracy: 0.4317358732223511


In [ ]:
# Generate predictions
y_pred = np.argmax(model.predict(X_test), axis=-1)

225/225 [==============================] - 20s 90ms/step


In [ ]:
# Generate classification report
target_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

       Angry       0.32      0.30      0.31       985
     Disgust       0.12      0.12      0.12       102
        Fear       0.28      0.26      0.27      1043
       Happy       0.64      0.65      0.64      1765
         Sad       0.32      0.41      0.36      1210
    Surprise       0.58      0.58      0.58       795
     Neutral       0.41      0.33      0.37      1278

    accuracy                           0.43      7178
   macro avg       0.38      0.38      0.38      7178
weighted avg       0.43      0.43      0.43      7178



In [ ]:
import pickle

# Assume 'model' is your trained EmoNet model
model_path = '/content/drive/MyDrive/EmoNetModel.pickle'

# Save the model as a pickle file
with open(model_path, 'wb') as f:
    pickle.dump(model, f)


In [ ]:
import pickle

# Load the model from the pickle file
with open(model_path, 'rb') as f:
    model = pickle.load(f)


In [ ]:
from google.colab import files
import pickle

# Assuming 'model' is the loaded model
# Save the loaded model as a pickle file
loaded_model_path = '/content/drive/MyDrive/LoadedModel.pickle'

with open(loaded_model_path, 'wb') as f:
    pickle.dump(model, f)

# Download the pickle file
files.download(loaded_model_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>